# Satria Data Hitam

# Deskripsi Projek

- Untuk keperluan mengefektifkan pengelolaan anggaran BPJS Kesehatan, maka pada lomba BDC Satria Data 2022 Anda ditantang untuk melakukan prediksi status pulang peserta berdasarkan data FKTP BPJS Kesehatan. 
- Bangun pemodelan untuk klasifikasi status pulang peserta menjadi “Sehat” dan “Belum_Sehat” dengan memanfaatkan data train. 


## Import Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

In [2]:
col = ['Nomor peserta',
'Nomor keluarga',
'Bobot',
'ID Kunjungan FKTP',
'Tanggal datang kunjungan FKTP',
'Tanggal pulang kunjungan FKTP',
'Provinsi FKTP',
'Kabupaten/Kota FKTP',
'Kepemilikan FKTP',
'Jenis FKTP',
'Tipe FKTP',
'Tingkat Pelayanan FKTP',
'Jenis Poli FKTP',
'Segmen Peserta saat akses layanan FKTP',
'Kode dan nama diagnosis ICD 10 (3 digit)',
'Kode diagnosis ICD 10 (3 digit)',
'Kode diagnosis (beragam 3-5 digit)',
'Nama diagnosis berasal dari kode diagnosis FKP15',
'Provinsi faskes tujuan rujukan',
'Kabupaten/Kota faskes tujuan rujukan',
'Kepemilikan faskes tujuan rujukan',
'Jenis faskes tujuan rujukan',
'Tipe faskes tujuan rujukan',
'Poli faskes tujuan rujukan',
'Jenis Kunjungan FKTP',
'Kelas Status Pulang Peserta']

In [3]:
train = pd.read_csv(r"C:\Users\herar\Downloads\train_fktp.txt", names=col, infer_datetime_format=True, parse_dates=[4,5])
test = pd.read_csv(r"C:\Users\herar\Downloads\test_fktp.txt", names=col[:-1], infer_datetime_format=True, parse_dates=[4,5])

## Pre-Processing Data

In [4]:
train1 = train.copy()
test1 = test.copy()

clean_data = [train1, test1]

### Exploratory Data Analysis

In [5]:
print(f'Dimensi Train:\n{train.shape}')
print('-'*10)
print(f'Dimensi Test:\n{test.shape}')

Dimensi Train:
(4056898, 26)
----------
Dimensi Test:
(1014225, 25)


In [6]:
train.info(show_counts=True)
print('-'*10)
train.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4056898 entries, 0 to 4056897
Data columns (total 26 columns):
 #   Column                                            Non-Null Count    Dtype         
---  ------                                            --------------    -----         
 0   Nomor peserta                                     4056898 non-null  float64       
 1   Nomor keluarga                                    4056898 non-null  int64         
 2   Bobot                                             4056898 non-null  float64       
 3   ID Kunjungan FKTP                                 4056898 non-null  object        
 4   Tanggal datang kunjungan FKTP                     4056898 non-null  datetime64[ns]
 5   Tanggal pulang kunjungan FKTP                     4056898 non-null  datetime64[ns]
 6   Provinsi FKTP                                     4056898 non-null  int64         
 7   Kabupaten/Kota FKTP                               4056898 non-null  int64         
 8   Ke

,Nomor peserta,Nomor keluarga,Bobot,ID Kunjungan FKTP,Tanggal datang kunjungan FKTP,Tanggal pulang kunjungan FKTP,Provinsi FKTP,Kabupaten/Kota FKTP,Kepemilikan FKTP,Jenis FKTP,Tipe FKTP,Tingkat Pelayanan FKTP,Jenis Poli FKTP,Segmen Peserta saat akses layanan FKTP,Kode dan nama diagnosis ICD 10 (3 digit),Kode diagnosis ICD 10 (3 digit),Kode diagnosis (beragam 3-5 digit),Nama diagnosis berasal dari kode diagnosis FKP15,Provinsi faskes tujuan rujukan,Kabupaten/Kota faskes tujuan rujukan,Kepemilikan faskes tujuan rujukan,Jenis faskes tujuan rujukan,Tipe faskes tujuan rujukan,Poli faskes tujuan rujukan,Jenis Kunjungan FKTP,Kelas Status Pulang Peserta
2578943,89344913.0,101593803,49.059406,1230400510,2020-05-01,2020-05-01,12,1218,3,1,3,1,13.0,2,9999,NaN,9999,9999,98,9998,98,98,98,98,2,Sehat
2411909,85035864.0,57183306,63.661671,1088352076,2019-12-23,2019-12-23,32,3201,9,2,1,1,13.0,5,9999,NaN,9999,9999,98,9998,98,98,98,98,2,Sehat
1598776,16271667.0,18145538,170.814987,1344076450,2020-10-08,2020-10-08,36,3673,3,1,3,1,12.0,2,9999,NaN,9999,9999,98,9998,98,98,98,98,2,Sehat


In [7]:
print(f'Jumlah sample duplicated dalam Train:\n{train1.duplicated().sum()}')
print('-'*10)
print(f'Jumlah sample duplicated dalam Test:\n{test1.duplicated().sum()}')

Jumlah sample duplicated dalam Train:
0
----------
Jumlah sample duplicated dalam Test:
0


In [8]:
print(f'Jumlah null values pada setiap kolom di Train:\n{train1.isnull().sum()}')
print('-'*10)
print(f'Jumlah null values pada setiap kolom di Test:\n{test.isnull().sum()}')

Jumlah null values pada setiap kolom di Train:
Nomor peserta                                             0
Nomor keluarga                                            0
Bobot                                                     0
ID Kunjungan FKTP                                         0
Tanggal datang kunjungan FKTP                             0
Tanggal pulang kunjungan FKTP                             0
Provinsi FKTP                                             0
Kabupaten/Kota FKTP                                       0
Kepemilikan FKTP                                          0
Jenis FKTP                                                0
Tipe FKTP                                                 0
Tingkat Pelayanan FKTP                                    0
Jenis Poli FKTP                                           1
Segmen Peserta saat akses layanan FKTP                    0
Kode dan nama diagnosis ICD 10 (3 digit)                  0
Kode diagnosis ICD 10 (3 digit)                     1

In [9]:
for i in col[:-1]:
    print(f'Jumlah unique value kolom {i} pada Test: {len(test[i].unique())}')

Jumlah unique value kolom Nomor peserta pada Test: 478035
Jumlah unique value kolom Nomor keluarga pada Test: 298556
Jumlah unique value kolom Bobot pada Test: 37316
Jumlah unique value kolom ID Kunjungan FKTP pada Test: 1014225
Jumlah unique value kolom Tanggal datang kunjungan FKTP pada Test: 731
Jumlah unique value kolom Tanggal pulang kunjungan FKTP pada Test: 735
Jumlah unique value kolom Provinsi FKTP pada Test: 34
Jumlah unique value kolom Kabupaten/Kota FKTP pada Test: 505
Jumlah unique value kolom Kepemilikan FKTP pada Test: 9
Jumlah unique value kolom Jenis FKTP pada Test: 6
Jumlah unique value kolom Tipe FKTP pada Test: 9
Jumlah unique value kolom Tingkat Pelayanan FKTP pada Test: 3
Jumlah unique value kolom Jenis Poli FKTP pada Test: 26
Jumlah unique value kolom Segmen Peserta saat akses layanan FKTP pada Test: 5
Jumlah unique value kolom Kode dan nama diagnosis ICD 10 (3 digit) pada Test: 1663
Jumlah unique value kolom Kode diagnosis ICD 10 (3 digit) pada Test: 1663
Jumlah

### Feature Engineering

In [10]:
for dataset in clean_data:
    dataset.loc[dataset['Provinsi faskes tujuan rujukan'] != 99, 'Dirujuk'] = 1
    dataset.loc[dataset['Provinsi faskes tujuan rujukan'] == 99, 'Dirujuk'] = 0
    dataset['Lama kunjungan FKTP'] = dataset['Tanggal pulang kunjungan FKTP']-dataset['Tanggal datang kunjungan FKTP']

### Data Cleaning

In [11]:
for dataset in clean_data:
    drop_col = ['Nomor peserta','Nomor keluarga','Bobot','Provinsi FKTP','Kabupaten/Kota FKTP','Kepemilikan FKTP','Provinsi faskes tujuan rujukan','Kabupaten/Kota faskes tujuan rujukan','Kepemilikan faskes tujuan rujukan','Tanggal datang kunjungan FKTP', 'Tanggal pulang kunjungan FKTP']

    dataset.drop(drop_col, axis=1, inplace=True)
    dataset.dropna()

### Feature Engineering

In [12]:
test1['ID Kunjungan FKTP'] = test1['ID Kunjungan FKTP'].astype(str)
test1['Tingkat Pelayanan FKTP'] = test1['Tingkat Pelayanan FKTP'].astype(str)
test1['Kode diagnosis ICD 10 (3 digit)'] = test1['Kode diagnosis ICD 10 (3 digit)'].astype(str)
test1['Kode diagnosis (beragam 3-5 digit)'] = test1['Kode diagnosis (beragam 3-5 digit)'].astype(str)
test1['Nama diagnosis berasal dari kode diagnosis FKP15'] = test1['Nama diagnosis berasal dari kode diagnosis FKP15'].astype(str)

In [13]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

for dataset in clean_data:
    dataset['ID Kunjungan FKTP'] = le.fit_transform(dataset['ID Kunjungan FKTP'])
    dataset['Tingkat Pelayanan FKTP'] = le.fit_transform(dataset['Tingkat Pelayanan FKTP'])
    dataset['Kode dan nama diagnosis ICD 10 (3 digit)'] = le.fit_transform(dataset['Kode dan nama diagnosis ICD 10 (3 digit)'])
    dataset['Kode diagnosis ICD 10 (3 digit)'] = le.fit_transform(dataset['Kode diagnosis ICD 10 (3 digit)'])
    dataset['Kode diagnosis (beragam 3-5 digit)'] = le.fit_transform(dataset['Kode diagnosis (beragam 3-5 digit)'])
    dataset['Nama diagnosis berasal dari kode diagnosis FKP15'] = le.fit_transform(dataset['Nama diagnosis berasal dari kode diagnosis FKP15'])
    dataset['Lama kunjungan FKTP'] = le.fit_transform(dataset['Lama kunjungan FKTP'])
    dataset['Jenis Poli FKTP'] = le.fit_transform(dataset['Jenis Poli FKTP'])
train1['Kelas Status Pulang Peserta'] = le.fit_transform(train1['Kelas Status Pulang Peserta'])

In [14]:
train1

,Nomor peserta,Nomor keluarga,Bobot,ID Kunjungan FKTP,Provinsi FKTP,Kabupaten/Kota FKTP,Kepemilikan FKTP,Jenis FKTP,Tipe FKTP,Tingkat Pelayanan FKTP,Jenis Poli FKTP,Segmen Peserta saat akses layanan FKTP,Kode dan nama diagnosis ICD 10 (3 digit),Kode diagnosis ICD 10 (3 digit),Kode diagnosis (beragam 3-5 digit),Nama diagnosis berasal dari kode diagnosis FKP15,Provinsi faskes tujuan rujukan,Kabupaten/Kota faskes tujuan rujukan,Kepemilikan faskes tujuan rujukan,Jenis faskes tujuan rujukan,Tipe faskes tujuan rujukan,Poli faskes tujuan rujukan,Jenis Kunjungan FKTP,Kelas Status Pulang Peserta,Dirujuk,Lama kunjungan FKTP
0,631092,50765420,5567,3946160,61,6108,3,1,4,0,10,2,1836,1836,0,0,98,9998,98,98,98,98,2,1,1.0,0
1,499613,26931775,646,2985799,51,5171,9,3,2,0,0,4,762,780,3609,3387,98,9998,98,98,98,98,1,0,1.0,0
2,722168,227368233,213,3070082,35,3516,9,2,1,0,2,4,1686,1774,9301,1878,98,9998,98,98,98,98,1,0,1.0,0
3,335919,62313678,40611,1820367,34,3402,3,1,4,0,0,5,610,625,2852,2717,98,9998,98,98,98,98,1,0,1.0,0
4,157633,113446188,212,3761344,35,3509,3,1,4,0,0,1,611,626,2854,3738,98,9998,98,98,98,98,1,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4056893,697652,91270403,3999,769255,35,3505,9,3,2,0,10,4,1836,1836,0,0,98,9998,98,98,98,98,2,1,1.0,0
4056894,162612,30859241,1023,2145196,35,3514,9,2,1,0,0,5,550,565,2528,1311,35,3514,9,2,14,99,1,0,1.0,0
4056895,46559,8859416,1711,3959916,11,1171,4,2,1,0,11,5,1836,1836,0,0,98,9998,98,98,98,98,2,1,1.0,0
4056896,805828,399491834,7257,1939555,74,7402,3,1,3,0,0,2,763,781,3612,2550,98,9998,98,98,98,98,1,0,1.0,0


## Modelling

In [15]:
X_train = train1.copy()
X_train.drop('Kelas Status Pulang Peserta', axis=1, inplace=True)
X_train.to_numpy()
Y_train = train1['Kelas Status Pulang Peserta'].copy()

X_test = test1.copy()

### Naive Bayes Algorithm

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB

In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
classifier = GaussianNB()
classifier.fit(X_train, Y_train)
Y_predNBA = classifier.predict(X_test)

In [19]:
Y_predNBA

array([1, 1, 1, ..., 0, 0, 0])

## Output

In [72]:
index =[]  
for n in range(1014225): 
    index.append(n+1)

In [73]:
ID_test = test['ID Kunjungan FKTP']

submission = {'':index,'FKP02': ID_test, 'Status':Y_predNBA}
sub = pd.DataFrame(submission)

In [74]:
sub

,,FKP02,Status
0,1,954386050,1
1,2,774246805,1
2,3,1202010273,1
3,4,1286640248,1
4,5,1285179242,1
...,...,...,...
1014220,1014221,248870919P000406,0
1014221,1014222,116000620P000034,0
1014222,1014223,160421019P000631,0
1014223,1014224,332670919P000083,0


In [76]:
sub.to_csv("submission.csv", index=False)